In [ ]:
import pandas as pd
import numpy as np
from konlpy.tag import Okt
import emoji
import re
from soynlp.normalizer import * #반복되는 자음 제거
from pykospacing import Spacing #띄어쓰기
import time
from tqdm.notebook import tqdm #for문 진행률 
from time import sleep #for문 진행 시간 
import requests #맞춤법 검사
import json #맞춤법 검사
from kiwipiepy import Kiwi #문장 분리2
from collections import Counter

okt = Okt()


In [ ]:
#의미를 가지고 있는 명사, 형용사, 동사를 konlpy 패키지를 사용하여 추출함.
#collections 모듈의 Counter 함수로 단어의 빈도를 구함.
#상위 200위 이내 단어를 수정 및 불용어 처리로 2차 정제함.

In [ ]:
#파일불르기
df = pd.read_csv("D:/2022jasago_news_url_title.csv", encoding='UTF-8')
title_list = list(df['title']) #자연어 전처리를 위해 본문을 리스트 형태로


In [ ]:
#불용어 제거_ re를 활용함
for i in range(len(title_list)):
    title_list[i] = re.sub('[^A-Za-z0-9가-힣]', ' ', string=str(title_list[i])) #한글, 영어, 숫자만 남기기
    title_list[i] = repeat_normalize(title_list[i], num_repeats=2) #ㅋㅋ, ㅎㅎ 등의 의미 없는 자음 삭제
    
df['cleaning'] = pd.DataFrame(title_list) #불용어 제거된 본문을 데이터 프레임으로

In [ ]:
#문장 분리
kiwi = Kiwi()

start = time.time()
split_list = [kiwi.split_into_sents(df['cleaning'][i]) for i in tqdm(range(len(df['cleaning'])))]
df['split_list'] = split_list
end = time.time()

print(f"{end - start:.5f} sec")

In [ ]:
#한 row
df2 = pd.DataFrame()

for i in tqdm(range(len(df['split_list']))):
    for j in range(len(df['split_list'][i])):
                                                            # 원래 기사의 순서를 index로 저장
        split_str= {'split_str': df['split_list'][i][j][0], 'index': [i]}
        split_df= pd.DataFrame(split_str)
        df2= pd.concat([df2,split_df])
        
df2.reset_index(drop=True, inplace=True)

In [ ]:
#맞춤법
modify_list = []

def matchum(a):
    for i in tqdm(range(len(a))):
        try:                                                                               
            response = requests.post('http://164.125.7.61/speller/results', data={'text1': a[i]})  # 맞춤법 검사 요청 (requests)    
            data = response.text.split('data = [', 1)[-1].rsplit('];', 1)[0] # 응답에서 필요한 내용 추출 (html 파싱)
            data = json.loads(data) # 파이썬 딕셔너리 형식으로 변환
            orgStr = [err['orgStr'] for err in data['errInfo']] #오류가 담긴 errinfo에서 고쳐야 되는 단어 리스트로 추출 
            modify = [err['candWord'] for err in data['errInfo']] #고쳐진 단어도 리스트로 추출
            splited = a[i] 

            #여러 개로 고쳐졌을 때 맨 처음 단어만 선택
            for j in range(len(modify)):
                if '|'in modify[j]: #여러 개로 고쳐질 경우 문자열이 '|'로 구분되어 있음
                    com = re.compile('\|')
                    many = com.search(modify[j])
                    modify[j]= modify[j][0:(many.span()[0])]
                #틀린 문장 고치기
                splited = splited.replace(orgStr[j], modify[j])

        except:      
            splited = a[i]

        modify_list.append(splited)

In [ ]:
#마춤 검사
matchum(df2['split_str'])

In [ ]:
#게시글별로
# 문자열로 변환
df2["modify_str"]=modify_list
df2

In [ ]:
df3 = df2.groupby('index').agg({'modify_str':' '.join})
df3

In [ ]:
df4 = pd.concat([df[['url', 'title', 'cleaning']], df3[['modify_str']]], axis=1)
df4

In [ ]:
#형태소
hyeong = []
for i in tqdm(range(len(df4))):
    a = okt.pos(df4['modify_str'][i], norm=True, stem=True)
    hyeong.append(a)
    

len(hyeong)

In [ ]:
#명사
noun_list = []
for i in tqdm(range(len(df))):
    a = okt.nouns(df4['modify_str'][i])
    noun_list.append(a)


In [ ]:
#동사와 형용사도 추가
for i in range(len(hyeong)):
    for a, b in hyeong[i]:
        if b == 'Verb':
            noun_list[i].append(a)
        elif b == 'Adjective':
            noun_list[i].append(a)

In [ ]:
noun_list

In [ ]:
words_df = pd.DataFrame({'words':noun_list})
words_df

In [ ]:
for i in tqdm(range(len(words_df))):
    words_df['words'][i] = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", "", str(words_df['words'][i]))
    words_df['words'][i] = ' '.join(words_df['words'][i].split())

In [ ]:
words_df

In [ ]:
words_df.to_csv("명사동사형용사.csv", index=False, encoding='UTF-8')

In [ ]:
#이중리스트 해제_어려움
def flatten(data):
    #변환된 값을 넣을 빈 리스트 선언
    result = []
    for i in data:#리스트 안에 리스트 요소의 타입을 확인할 반복문
        if type(i) == list: #리스트 안에 요소의 자료형 판별 조건문
            result += flatten(i) #리스트 데이터가 리스트일 때 재귀함수로 들어가서 인덱스 꺼내기
        else:
            result +=[i] #리스트의 데이터가 리스트형이 아닐 때 result에 요소 넣기
    return result

In [ ]:
# 단어 빈도수 추출
words_list = flatten(noun_list)
words_list

In [ ]:
word_count = Counter(words_list)
print(len(word_count))
word_n_freq = word_count.most_common(2984)
word_n_freq


In [ ]:
# 단어와 빈도 뽑아서 파일로 저장
WORD = []
FREQ = []

for i, j in word_n_freq:
    WORD.append(i)
    FREQ.append(j)

word_n_freq_df = pd.DataFrame({'단어':WORD, '빈도':FREQ})
word_n_freq_df.to_excel("2022jasago_news_url_title.csv", encoding='UTF-8')


In [ ]:
word_n_freq_df